### I will learn about chess move prediction using CNN to get the idea of how to represent peices , squares and move. Then my motive is to use self attention mechanism or transformer to do the same and check how is my chess prediction model is improved

In [ ]:
!pip install kaggle -q

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! kaggle datasets download arevel/chess-games
! unzip -qq '/content/chess-games.zip'

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 99% 1.43G/1.45G [00:33<00:00, 79.1MB/s]
100% 1.45G/1.45G [00:33<00:00, 45.9MB/s]


In [ ]:
! pip install chess -q
! pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 503.4 kB/s eta 0:00:00


In [ ]:
import chess
import numpy as np
import re
import pandas as pd
import gc
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import torch.nn as nn
from einops import rearrange
from torch import einsum


device =torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

letter_2_num = {'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
num_2_letter = {0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h'}

In [ ]:
def create_rep_layer(board,type):
  """ give matrix of 1x8x8
  """

  s = str(board)
  s = re.sub(f'[^{type}{type.upper()} \n]','.',s)
  s = re.sub(f'{type}','-1',s)
  s = re.sub(f'{type.upper()}','1',s)
  s = re.sub(f'\.','0',s)

  board_mat = []
  for row in s.split('\n'):
    row = row.split(' ')
    row = [int(x) for x in row]
    board_mat.append(row)

  return np.array(board_mat)



In [ ]:
def board_2_rep(board):
  """ create rep layers 6 times
  """
  pieces = ['p','r','n','b','q','k']
  layers = []

  for piece in pieces:
    layers.append(create_rep_layer(board,piece))
  board_rep = np.stack(layers)

  return board_rep

In [ ]:
def mov_2_rep(move,board):
  """
  gives 2x8x8 with one from, to layers will have only one piece position( the piece that is being moved).
  """
  board.push_san(move).uci()

  move = str(board.pop())

  from_output_layer = np.zeros((8,8))
  from_row = 8 - int(move[1])
  from_column = letter_2_num[move[0]]
  from_output_layer[from_row,from_column] = 1


  to_output_layer = np.zeros((8,8))
  to_row = 8 - int(move[3])
  to_column = letter_2_num[move[2]]
  to_output_layer[to_row,to_column] = 1

  return np.stack([from_output_layer,to_output_layer])

In [ ]:
def create_move_list(s):
  """ moves logging
  """
  return re.sub('\d*\. ','',s).split(' ')[:-1]

In [ ]:
def check_mate_single(board):
  """
  make check mate, if it is in one move
  """
  board = board.copy()
  legal_moves = list(board.legal_moves)

  for move in legal_moves:
    board.push_uci(str(move))
    if board.is_checkmate():
      move = board.pop()
      return move
    _ = board.pop()

In [ ]:
chess_data_raw = pd.read_csv('/content/chess_games.csv',usecols=['AN','WhiteElo'])
chess_data = chess_data_raw[chess_data_raw['WhiteElo']>2000]

del chess_data_raw

gc.collect()

13

In [ ]:
chess_data = chess_data[['AN']]
chess_data = chess_data[-chess_data['AN'].str.contains('{')]
chess_data = chess_data[chess_data['AN'].str.len() > 20]
#
print(chess_data.shape[0])

883376


In [ ]:
class ChessDataset(Dataset):

  def __init__(self,games):
    super(ChessDataset,self).__init__()
    self.games = games
    #suffle here, one time

  def __len__(self):
    return 40_000

  def __getitem__(self,index):

    game_i = np.random.randint(self.games.shape[0])
    random_game = chess_data['AN'].values[game_i]   #use index instead of picking random
    moves = create_move_list(random_game)
    game_state_i = np.random.randint(len(moves)-1)

    next_move = moves[game_state_i]
    moves = moves[:game_state_i]

    board = chess.Board()

    for move in moves:
      board.push_san(move)

    x = board_2_rep(board)
    y = mov_2_rep(next_move, board)

    if game_state_i % 2 == 1:
      x *= -1  # based on white or black ( kind of like swapping picies if it's the black's turn)

    x = torch.Tensor(x) #1x6x8x8 , board state at a random time
    y = torch.Tensor(y)#1x2x8x8  , next move ( to be predicted)

    return x,y

In [ ]:
data_train = ChessDataset(chess_data['AN'])

data_train_loader = DataLoader(data_train, batch_size = 1, shuffle = True, drop_last = True)

In [ ]:
class module(nn.Module):

  def __init__(self,hidden_size):
    super(module, self).__init__()

    self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride = 1, padding = 1)

    self.bn1 = nn.BatchNorm2d(hidden_size)
    self.bn2 = nn.BatchNorm2d(hidden_size)

    self.activation1 = nn.SELU()
    self.activation2 = nn.SELU()


  def forward(self,x):
    x_input = torch.clone(x)
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.activation1(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = x + x_input
    x = self.activation2(x)

    return x


In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, query_dim, context_dim=None, heads=2, dim_head=64, dropout=0.3):
        super().__init__()

        inner_dim = dim_head * heads

        self.scale = dim_head ** -0.5
        self.heads = heads

        self.to_q = nn.Linear(query_dim, inner_dim, bias=False)     ##(64 to 8*64)
        self.to_k = nn.Linear(query_dim, inner_dim, bias=False)
        self.to_v = nn.Linear(query_dim, inner_dim, bias=False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, query_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x, context=None, mask=None, timesteps =None):
        x = x.view(x.shape[0],x.shape[1],-1)    #[32,1,64]

        batch_size,num_channels ,seq_len = x.size()

        positional_encoding = self.get_positional_encoding(seq_len, num_channels, x.device)   ###check

        x = x + positional_encoding.permute(0,2,1)





        h = self.heads      ## shape of x is [1,6,64]

        q = self.to_q(x)    ## shape q is [1,1,8*64]
        context = x
        k = self.to_k(context)    # shape k is [1,1,8*64]
        v = self.to_v(context)    # shape v is [1,1,8*64]

        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) d n', h=h), (q, k, v))     ###(8,64,1)

        sim = einsum('b i d, b j d -> b i j', q, k) * self.scale

        attn = sim.softmax(dim=-1)      ##attn shape is [8,64,64]     (b*head, height*width, channels)


        out = einsum('b i j, b j d -> b i d', attn, v)    #[8,64,1]
        out = rearrange(out, '(b h) n d -> b d (h n)', h=h)   #[1,6,64]
        out = self.to_out(out)

        out = rearrange(out,'b c (h w) -> b c h w', w= 8)

        return out



    def get_positional_encoding(self, seq_len, dim, device):
          positional_encoding = torch.zeros(seq_len, dim).to(device)
          position = torch.arange(0, seq_len, dtype=torch.float32).unsqueeze(1)
          div_term = torch.exp(torch.arange(0, dim, 2, dtype=torch.float32) * -(torch.log(torch.tensor(10000.0)) / dim))
          positional_encoding[:, 0::2] = torch.sin(position * div_term)
          positional_encoding[:, 1::2] = torch.cos(position * div_term)
          return positional_encoding.unsqueeze(0)




In [ ]:
class ChannelSelector(nn.Module):
    def __init__(self):
        super(ChannelSelector, self).__init__()
        self.conv1 = nn.Conv2d(6, 6, kernel_size=3)
        self.fc = nn.Linear(6 * 6 * 6, 6)
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(x.size(0), -1)
        x = torch.softmax(self.fc(x), dim=1)
        x = torch.argmax(x,dim=1)
        return x

In [ ]:
class ChessNet(nn.Module):

  def __init__(self,hidden_layers = 4, hidden_size= 10):
    super(ChessNet, self).__init__()

    self.cs = ChannelSelector()

    self.attention = SelfAttention(64)

    self.hidden_layers = hidden_layers

    """To choose the layer for the move """

    # self.to_move = nn.Conv2d(6)



    self.input_layer = nn.Conv2d(1,hidden_size,3, stride =1, padding=1)

    self.module_list = nn.ModuleList([module(hidden_size) for i in range(hidden_layers)])

    self.output_layer = nn.Conv2d(hidden_size,1,3,stride=1,padding=1)

    self.channel_embedding = nn.Embedding(6, 64)



  def forward(self,x):

    idx = self.cs(x)

    to_choose = torch.cat([x[i, c_i, :, :][None].unsqueeze(0) for i,c_i in enumerate(idx)], dim=0)


    # positional_encoding = self.channel_embedding(idx).view(x.shape[0],1,8,8)*0.0001

    x = self.attention(to_choose)    ##[1,8,8]

    print(x)

    x = self.input_layer(x)
    x = F.relu(x)

    for i in range(self.hidden_layers):
      x = self.module_list[i](x)

    x = self.output_layer(x)

    x = torch.cat((to_choose,x),dim=1)

    return x



In [ ]:
def train(model,device,data_train_loader,optimizer,loss1,loss2,epoch):

  model.train()
  epoch_loss = 0


  for batch_idx, (move,next_move) in enumerate(data_train_loader):

    ## move = [1,6,8,8] and next_move = [1,2,8,8]
    move = move.to(device)
    next_move = next_move.to(device)

    optimizer.zero_grad()

    predicted_move = model(move)

    loss1 = metric_from(predicted_move[:,0,:],next_move[:,0,:])
    loss2 = metric_to(predicted_move[:,1,:],next_move[:,1,:])



    loss = loss1 + loss2

    loss.backward()

    optimizer.step()

    epoch_loss += loss.item() * move.shape[0]

  print(f"Train Average Loss: {epoch_loss/len(data_train_loader.dataset):.2f}")




In [ ]:

model = ChessNet(hidden_layers=2).to(device)
metric_from = nn.CrossEntropyLoss()
metric_to = nn.CrossEntropyLoss()
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)




In [ ]:
for epoch in range(1, num_epochs+1):
    print(f"\nEpoch: {epoch}/{num_epochs}")

    train(model, device, data_train_loader, optimizer, metric_from , metric_to, epoch)


Epoch: 1/10
tensor([[[[ -8.7189,   1.6139,  -3.9768,   2.6787,   0.8860,   3.3246,  -0.0000,
            -7.0376],
          [  4.7504,  -2.8703,  -8.1510,   5.2914,   0.0000,   1.6659,   3.6140,
            19.7969],
          [  1.9029,  10.7786,   2.7100,  -0.7781,   2.0320,  -0.0000,  -1.8744,
            -0.0000],
          [  0.8403,   0.0000,  13.8324,  -0.5105,  -0.0000,  -8.0336,   8.0251,
            -6.0687],
          [  0.9466,  -1.1162,  -4.4099,  -5.5497,  -0.0000,   5.0595,  -4.6509,
             0.0000],
          [ -0.0000,  -4.7139,   1.3269,  -4.0985,  -6.7631,   2.9866,   0.0000,
            -0.0000],
          [ -0.0000,  -0.0000, -12.6693,  13.9759,  -8.5468,   0.0000,   2.9224,
           -10.7953],
          [  0.0000,  -0.0000,   2.7129,  14.8112,  13.4445,   2.5008,   1.4615,
            -4.4444]]]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
tensor([[[[  0.0000,  -4.5208,  10.9192,  -6.0468,  -0.0000,   5.2472,   7.3296,
             0.0000],
       

KeyboardInterrupt: ignored

In [ ]:
model

ChessNet(
  (cs): ChannelSelector(
    (conv1): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1))
    (fc): Linear(in_features=216, out_features=6, bias=True)
  )
  (attention): SelfAttention(
    (to_q): Linear(in_features=64, out_features=128, bias=False)
    (to_k): Linear(in_features=64, out_features=128, bias=False)
    (to_v): Linear(in_features=64, out_features=128, bias=False)
    (to_out): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): Dropout(p=0.0, inplace=False)
    )
  )
  (input_layer): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (module_list): ModuleList(
    (0-1): 2 x module(
      (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [ ]:
inputs, outputs = next(iter(data_train_loader))
inputs = inputs.to(device)
outputs = outputs.to(device)

z =model(inputs)
max_value = torch.amax(z.view(-1,64),dim=1)
max_value=max_value.to(device)

print(z[:,1,:].unsqueeze(1))

# from_move = max_value[0]*torch.ones((1,1,8,8),device=device)
to_move = max_value[1]*torch.ones((1,1,8,8),device=device)

(z == torch.cat((z[:,0,:].unsqueeze(1),to_move), dim=1))[0].float()

tensor([[[[ 6.2886,  7.7477,  7.6585,  7.7687,  7.7999,  7.9049,  7.8011,
            7.0169],
          [ 7.7614, 10.5252, 10.3881, 10.4369, 10.4054, 10.5867, 10.2847,
            8.8293],
          [ 7.8618, 10.7333, 10.7321, 10.7465, 10.7053, 10.8669, 10.4694,
            9.0190],
          [ 7.8322, 10.7213, 10.6716, 10.7217, 10.7302, 10.8749, 10.4347,
            8.9252],
          [ 7.8438, 10.7505, 10.7281, 10.7456, 10.7263, 10.8559, 10.4214,
            8.9149],
          [ 7.7683, 10.5672, 10.2840, 10.3454, 10.4050, 10.2733, 10.0150,
            8.6715],
          [ 7.2976,  9.9546,  9.8522, 10.0227,  9.9429,  9.6764, 10.3315,
            8.6347],
          [ 6.3758,  8.0214,  7.9200,  7.8569,  7.8710,  7.6621,  7.8491,
            6.4152]]]], device='cuda:0', grad_fn=<UnsqueezeBackward0>)


tensor([[[1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0')

In [ ]:
inputs[0]

tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.],
         [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
         [ 0.,  0., -1., -1.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0., -1., -1., -1.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0

In [ ]:
outputs[0]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0')